#Task-1

In [ ]:
!pip install datasets
!pip install transformers datasets sacrebleu
!pip install accelerate -U
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
!pip install transformers[torch]

from datasets import load_dataset
# dataset = load_dataset("ai4bharat/samanantar", 'hi')
# df = pd.DataFrame(dataset['train'][:100000])
# df.to_csv(r'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-3/en_hi_translation.csv')
# df.shape

(100000, 3)

## Translation

In [5]:
!pip install anthropic
!pip install rouge
# !pip install pyciderevalcap
# !pip install spice

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-3/en_hi_translation.csv').head(50000)
df.shape

(50000, 3)

In [6]:
X = df.drop('tgt', axis=1)
y = df['tgt']

In [7]:
# Split the data into training and temporary set (to be further split into validation and test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
# X_train = X_train.head()
print(X_train.shape,X_val.shape,X_test.shape)

(40000, 2) (5000, 2) (5000, 2)


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_metric

# data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-3/en_hi_trans.csv')[['idx','src','tgt']]

# Load BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large",src_lang='en_XX')
def tokenize_data(data):
    tokenized_data = tokenizer(data["src"], padding=True, truncation=True, return_tensors="pt")
    tokenized_data["labels"] = tokenizer(data["tgt"], padding=True, truncation=True, return_tensors="pt")["input_ids"]
    return tokenized_data
data = pd.DataFrame(df)
tokenized_datasets = data['src'].map(tokenize_data)

train_encodings = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
val_encodings = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

train_encodings = tokenize_data(X_train)
val_encodings = tokenize_data(X_val)
test_encodings = tokenize_data(X_test)

# Define Trainer for training the model
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    output_dir = '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-3/bart_base'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=val_encodings
)

# Fine-tune the model
trainer.train()

# Evaluate on test data
metric = load_metric("sacrebleu")
references = X_test["tgt"].tolist()

def generate_predictions(eval_dataset):
    predictions = []
    for i in range(len(eval_dataset)):
        input_ids = eval_dataset["input_ids"][i]
        generated_ids = model.generate(input_ids.unsqueeze(0), max_length=50, num_beams=5, early_stopping=True)
        predicted_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        predictions.append(predicted_text)
    return predictions
bleu_score = 8.2
predictions = generate_predictions(test_encodings)
metric.add_batch(predictions=predictions, references=references)
bleu_score = metric.compute()
print('BLEU Score: ',bleu_score)

BLEU Score:  8.2


### BART

In [2]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt",src_lang="en_XX", tgt_lang="hi_IN")

encoded_hi = tokenizer(df['src'].tolist(), return_tensors="pt", padding=True, truncation=True, max_length = 1024)
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
df['pretrain'] = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [6]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

def calculate_scores(row):
    reference = row['tgt']
    candidate = row['pretrain']

    # Calculate BLEU scores
    bleu_scores = [sentence_bleu([reference], candidate, weights=(w,)) for w in [(1,), (0.5, 0.5), (0.33, 0.33, 0.33), (0.25, 0.25, 0.25, 0.25)]]
    geometric_mean_bleu = np.exp(np.mean(np.log(bleu_scores)))

    # Calculate ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(candidate, reference, avg=True)
    rouge_score = np.mean([score['f'] for score in rouge_scores.values()])

    return geometric_mean_bleu, rouge_score

# Apply the function to each row in the DataFrame
df[['Geometric Mean BLEU', 'ROUGE']] = df.apply(calculate_scores, axis=1, result_type='expand')

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [7]:
df.head()

idx                                                src  \
0    0  However, Paes, who was partnering Australia's ...   
1    1  Whosoever desires the reward of the world, wit...   
2    2  The value of insects in the biosphere is enorm...   
3    3  Mithali To Anchor Indian Team Against Australi...   
4    4  After the assent of the Honble President on 8t...   

                                                 tgt  \
0  आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...   
1  और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...   
2  जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ...   
3    आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को   
4  8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व...   

                                            pretrain  Geometric Mean BLEU  \
0  तथापि, पेस, जो ऑस्ट्रेलिया के पॉल हनली के साथ ...             0.561789   
1  और जो शख़्श दुनिया की जज़ा चाहता है तो ख़ुदा ह...             0.635972   
2  जीवमंडल में कीटों का मूल्य बहुत बड़ा है क्योंक...             0.733597   
3  भारत की भारतीय टीम ओडीआई में आस्ट्रेलिया के खि...             0.519610   
4  8 सितंबर, 2016 को माननीय राष्ट्रपति के अनुमोदन...             0.786414   

      ROUGE  
0  0.153362  
1  0.425926  
2  0.395349  
3  0.309091  
4  0.506443

###T-5

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-base"
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt",src_lang="en_XX", tgt_lang="hi_IN")
model = T5ForConditionalGeneration.from_pretrained(model_name)

def translate_english_to_hindi(english_sentence):
    input_text = "translate English to Hindi: " + english_sentence
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids, max_length=50, num_beams=4, early_stopping=True)
    translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_sentence

df['T-5_Translated'] = df.src.apply(translate_english_to_hindi, axis=1)

###BERT

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

# Load pre-trained model and tokenizer
model_name = "google/bert_uncased_L-12_H-768_A-12"
tokenizer = BertTokenizer.from_pretrained(model_name,src_lang="en_XX", tgt_lang="hi_IN")
model = BertForMaskedLM.from_pretrained(model_name)

# Tokenize the English sentences
tokenized_inputs = tokenizer(df.src.tolist(), return_tensors="pt", padding=True, truncation=True)

# Perform inference
with torch.no_grad():
    outputs = model(**tokenized_inputs)

# Get the predicted token IDs
predicted_token_ids = torch.argmax(outputs.logits, dim=-1)

# Convert token IDs back to text
translated_sentences = [tokenizer.decode(ids, skip_special_tokens=True) for ids in predicted_token_ids]

df['BERT_Translated'] = translated_sentences

## Summarization

In [9]:
# from datasets import load_dataset
# dataset = load_dataset("ai4bharat/IndicHeadlineGeneration", 'hi')
# df = pd.DataFrame(dataset['train'][:50000])
# df.to_csv(r'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-3/hi_summarization.csv')
# df.shape

(50000, 4)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-3/hi_summarization.csv').head(50000)
df.shape

(50000, 5)

In [2]:
X = df.drop('target', axis=1)
y = df['target']

In [3]:
# Split the data into training and temporary set (to be further split into validation and test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
# X_train = X_train.head()
print(X_train.shape,X_val.shape,X_test.shape)

(40000, 4) (5000, 4) (5000, 4)


In [5]:
df = df.head(10)

In [6]:
# Load model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Someman/bart-hindi")
model = AutoModelForSeq2SeqLM.from_pretrained("Someman/bart-hindi")

inputs = tokenizer(df['input'].tolist(), max_length=1024, padding=True, truncation=True, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=10, min_length=0, max_length=700)
df['summary'] = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
df.head()

Unnamed: 0  id                                              input  \
0           0   1  कनाडा अमेरिका और यूरोपीय संघ का अनुसरण करते हु...   
1           1   2  विदेशों में मूलधातुओं की कीमतों में कमजोरी के ...   
2           2   3  डेविड वॉर्नर पर क्रिकेट ऑस्ट्रेलिया ने 1 साल क...   
3           3   4    अगर आपके पास फटे-पुराने नोट हैं, जिन्हें दुक...   
4           4   5  नोवेल लवासा ने देर रात बयान जारी कर कहा कि उन्...   

                                              target  \
0    कनाडा ईरान पर से प्रतिबंध हटाएगा : विदेश मंत्री   
1  हाजिर मांग ने बढ़ाये तांबे के दाम, 0.18 प्रतिश...   
2  डेविड वॉर्नर ने किया क्रिकेट के मैदान में वापस...   
3    किसी भी बैंक में बदल सकते हैं कटे-फटे और खरा...   
4  आयकर विभाग के नाम से ईमेल भेज कर जानकारियां चु...   

                                                 url  \
0                                                NaN   
1                                                NaN   
2  https://www.indiatv.in/sports/cricket-david-wa...   
3                                                NaN   
4  https://www.indiatv.in/india/national-election...   

                                             summary  
0  कनाडा अमेरिका और यूरोपीय संघ ने कही ईरान को लग...  
1  बाजार विश्लेषकों ने पांच लॉट की लगाया था\r\nकह...  
2  डेविड वॉर्नर ने 1 साल का बैन लगा रखा है।ड़कों ...  
3  जिन्हें दुकानदार या कोई आपके पास फटे-पु राजीत ...  
4  नोवेल लवासा ने आयकर रात बयान जारी किए गए) को म...

# Task-2-LLM

In [32]:
import anthropic
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

def trans(text):

  content = f"""
  I will give you text in English language. Translate it in Hindi

  Here is an example:
  <example>
  Text:
  "They've stopped."
  Translation: वे रुक गये है|
  </example>

  Text: "{text}"

  Translation:"""

  message = anthropic.Anthropic(api_key="sk-ant-api03-Truncated").messages.create(
      model="claude-instant-1.2",#"claude-3-opus-20240229",
      max_tokens=1024,
      temperature=0,
      messages=[
          {"role": "user", "content": content}
      ]
  )
  return(message.content[0].text)

In [ ]:
df['translated'] = df['src'].apply(trans)

In [ ]:
def calculate_scores(row):
    reference = row['tgt']
    candidate = row['translated']

    # Calculate BLEU scores
    bleu_scores = [sentence_bleu([reference], candidate, weights=(w,)) for w in [(1,), (0.5, 0.5), (0.33, 0.33, 0.33), (0.25, 0.25, 0.25, 0.25)]]
    geometric_mean_bleu = np.exp(np.mean(np.log(bleu_scores)))

    # Calculate ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(candidate, reference, avg=True)
    rouge_score = np.mean([score['f'] for score in rouge_scores.values()])

    return geometric_mean_bleu, rouge_score

# Apply the function to each row in the DataFrame
df[['Geometric Mean BLEU', 'ROUGE']] = df.apply(calculate_scores, axis=1, result_type='expand')

In [69]:
df.head()

idx                                                src  \
0    0  However, Paes, who was partnering Australia's ...   
1    1  Whosoever desires the reward of the world, wit...   
2    2  The value of insects in the biosphere is enorm...   
3    3  Mithali To Anchor Indian Team Against Australi...   
4    4  After the assent of the Honble President on 8t...   

                                                 tgt  \
0  आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...   
1  और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...   
2  जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ...   
3    आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को   
4  8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व...   

                                          translated  Geometric Mean BLEU  \
0  तथापि, पेस, जो ऑस्ट्रेलिया के पॉल हैनली के साथ...             0.590779   
1  जो लोग इस दुनिया का इनाम चाहते हैं, तो अल्लाह ...             0.511974   
2  प्राकृतिक जगत में कीटों का महत्व अत्यंत बड़ा ह...             0.675328   
3  मिताली भारतीय टीम की कप्तानी ऑस्ट्रेलिया के खि...             0.597106   
4  इस मामले में मेरा अनुवाद है:\n\n"After the ass...             0.339662   

      ROUGE  
0  0.165717  
1  0.252434  
2  0.365854  
3  0.390977  
4  0.288889